In [ ]:
!pip install emot
!pip install unidecode

# for bert model
!pip install transformers
!pip install symspellpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 18 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.4 MB/s 
     |████████████████████████████████| 596 kB 17.3 MB/s 
     |████████████████████████████████| 6.6 MB 29.1 MB/s 
     |████████████████████████████████| 101 kB 8.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.0 MB/s 
     |████████████████████████████████| 125 kB 31.0 MB/s

## Import statements

In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import TfidfVectorizer

import re
import pandas as pd
from nltk.corpus import stopwords

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

import string
import pickle

import emot
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import random

# to check spelling
import pkg_resources
from symspellpy import SymSpell, Verbosity

# for bert model
from transformers import BertTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
data_path = "/content/drive/MyDrive/data_mmai/"

## Preprocessing

### Reading input

In [ ]:
data_df = pd.read_csv(data_path + "Suicide_Detection.csv")

In [ ]:
data_df

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide
...,...,...,...
232069,348103,If you don't like rock then your not going to ...,non-suicide
232070,348106,You how you can tell i have so many friends an...,non-suicide
232071,348107,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide
232072,348108,The usual stuff you find hereI'm not posting t...,suicide


In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data_df['class'])
norm_labels = le.transform(data_df['class'])

In [ ]:
data_df = data_df.drop('Unnamed: 0', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
data_df['encoded_class'] = norm_labels

In [ ]:
data_df

,text,class,encoded_class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,1
1,Am I weird I don't get affected by compliments...,non-suicide,0
2,Finally 2020 is almost over... So I can never ...,non-suicide,0
3,i need helpjust help me im crying so hard,suicide,1
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,1
...,...,...,...
232069,If you don't like rock then your not going to ...,non-suicide,0
232070,You how you can tell i have so many friends an...,non-suicide,0
232071,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide,0
232072,The usual stuff you find hereI'm not posting t...,suicide,1


### Converting to unicode

In [ ]:
data_df['uni_text'] = data_df['text'].apply(lambda x: unidecode(x))

In [ ]:
data_df

,text,class,encoded_class,uni_text
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,1,Ex Wife Threatening SuicideRecently I left my ...
1,Am I weird I don't get affected by compliments...,non-suicide,0,Am I weird I don't get affected by compliments...
2,Finally 2020 is almost over... So I can never ...,non-suicide,0,Finally 2020 is almost over... So I can never ...
3,i need helpjust help me im crying so hard,suicide,1,i need helpjust help me im crying so hard
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,1,"I'm so lostHello, my name is Adam (16) and I'v..."
...,...,...,...,...
232069,If you don't like rock then your not going to ...,non-suicide,0,If you don't like rock then your not going to ...
232070,You how you can tell i have so many friends an...,non-suicide,0,You how you can tell i have so many friends an...
232071,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide,0,pee probably tastes like salty tea!! can someo...
232072,The usual stuff you find hereI'm not posting t...,suicide,1,The usual stuff you find hereI'm not posting t...


### Converting Emojis & Removing Punctuation

In [ ]:
def convert_emojis(text):
    rem_punc_text = text.translate(str.maketrans('', '', string.punctuation))
    text_split = list(rem_punc_text)
    for i, item in enumerate(text_split):
        if item in UNICODE_EMOJI.keys():
            text_split[i] = " " + UNICODE_EMOJI[item].replace(",", "").replace(":", "").replace("_", " ") + " "

    conv_text = "".join(text_split).strip()
    return conv_text

In [ ]:
# 29 seconds
data_df['conv_emojis'] = data_df['uni_text'].apply(lambda x: convert_emojis(x))

In [ ]:
data_df

,text,class,encoded_class,uni_text,conv_emojis
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,1,Ex Wife Threatening SuicideRecently I left my ...,Ex Wife Threatening SuicideRecently I left my ...
1,Am I weird I don't get affected by compliments...,non-suicide,0,Am I weird I don't get affected by compliments...,Am I weird I dont get affected by compliments ...
2,Finally 2020 is almost over... So I can never ...,non-suicide,0,Finally 2020 is almost over... So I can never ...,Finally 2020 is almost over So I can never hea...
3,i need helpjust help me im crying so hard,suicide,1,i need helpjust help me im crying so hard,i need helpjust help me im crying so hard
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,1,"I'm so lostHello, my name is Adam (16) and I'v...",Im so lostHello my name is Adam 16 and Ive bee...
...,...,...,...,...,...
232069,If you don't like rock then your not going to ...,non-suicide,0,If you don't like rock then your not going to ...,If you dont like rock then your not going to g...
232070,You how you can tell i have so many friends an...,non-suicide,0,You how you can tell i have so many friends an...,You how you can tell i have so many friends an...
232071,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide,0,pee probably tastes like salty tea!! can someo...,pee probably tastes like salty tea can someone...
232072,The usual stuff you find hereI'm not posting t...,suicide,1,The usual stuff you find hereI'm not posting t...,The usual stuff you find hereIm not posting th...


### Converting to lowercase

In [ ]:
data_df['lower_text'] = data_df['conv_emojis'].apply(lambda x: x.lower())

In [ ]:
data_df

,text,class,encoded_class,uni_text,conv_emojis,spell_checked,lower_text
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,1,Ex Wife Threatening SuicideRecently I left my ...,Ex Wife Threatening SuicideRecently I left my ...,of life threatening SuicideRecently a left my ...,ex wife threatening suiciderecently i left my ...
1,Am I weird I don't get affected by compliments...,non-suicide,0,Am I weird I don't get affected by compliments...,Am I weird I dont get affected by compliments ...,pm a weird a done get affected by compliments ...,am i weird i dont get affected by compliments ...
2,Finally 2020 is almost over... So I can never ...,non-suicide,0,Finally 2020 is almost over... So I can never ...,Finally 2020 is almost over So I can never hea...,finally 2020 is almost over to a can never hea...,finally 2020 is almost over so i can never hea...
3,i need helpjust help me im crying so hard,suicide,1,i need helpjust help me im crying so hard,i need helpjust help me im crying so hard,i need helpjust help me in crying so hard,i need helpjust help me im crying so hard
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,1,"I'm so lostHello, my name is Adam (16) and I'v...",Im so lostHello my name is Adam 16 and Ive bee...,pm so costello my name is adam of and ave been...,im so losthello my name is adam 16 and ive bee...
...,...,...,...,...,...,...,...
232069,If you don't like rock then your not going to ...,non-suicide,0,If you don't like rock then your not going to ...,If you dont like rock then your not going to g...,of you done like rock then your not going to g...,if you dont like rock then your not going to g...
232070,You how you can tell i have so many friends an...,non-suicide,0,You how you can tell i have so many friends an...,You how you can tell i have so many friends an...,you how you can tell i have so many friends an...,you how you can tell i have so many friends an...
232071,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide,0,pee probably tastes like salty tea!! can someo...,pee probably tastes like salty tea can someone...,pee probably tastes like salty tea can someone...,pee probably tastes like salty tea can someone...
232072,The usual stuff you find hereI'm not posting t...,suicide,1,The usual stuff you find hereI'm not posting t...,The usual stuff you find hereIm not posting th...,the usual stuff you find here not posting this...,the usual stuff you find hereim not posting th...


### Checking spelling

In [ ]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)

In [ ]:
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [ ]:
# lookup suggestions for single-word input strings
# max edit distance per lookup
# (max_edit_distance_lookup <= max_dictionary_edit_distance)

def check_spelling(sentence):
  words = sentence.split(" ")
  new_sentence_list = []
  for word in words:
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
    if len(suggestions) > 0:
      new_sentence_list.append(suggestions[0].term)
    else:
      new_sentence_list.append(word)
  new_sentence = ' '.join(new_sentence_list)
  return new_sentence

In [ ]:
# suggestions = sym_spell.lookup("ex", Verbosity.CLOSEST, max_edit_distance=1)
# for sug in suggestions:
#   print(sug)

sex, 1, 320105999
exp, 1, 16394663
ext, 1, 11830447
tex, 1, 7342192
rex, 1, 4659516
hex, 1, 4217438
exc, 1, 3315237
lex, 1, 2424291
mex, 1, 1431119
vex, 1, 287473
exr, 1, 62565
kex, 1, 28228


In [ ]:
data_df['spell_checked'] = data_df['lower_text'].apply(lambda x: check_spelling(x))

In [ ]:
data_df

,text,class,encoded_class,uni_text,conv_emojis,spell_checked,lower_text
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,1,Ex Wife Threatening SuicideRecently I left my ...,Ex Wife Threatening SuicideRecently I left my ...,sex wife threatening suiciderecently i left my...,ex wife threatening suiciderecently i left my ...
1,Am I weird I don't get affected by compliments...,non-suicide,0,Am I weird I don't get affected by compliments...,Am I weird I dont get affected by compliments ...,am i weird i done get affected by compliments ...,am i weird i dont get affected by compliments ...
2,Finally 2020 is almost over... So I can never ...,non-suicide,0,Finally 2020 is almost over... So I can never ...,Finally 2020 is almost over So I can never hea...,finally 2020 is almost over so i can never hea...,finally 2020 is almost over so i can never hea...
3,i need helpjust help me im crying so hard,suicide,1,i need helpjust help me im crying so hard,i need helpjust help me im crying so hard,i need helpjust help me in crying so hard,i need helpjust help me im crying so hard
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,1,"I'm so lostHello, my name is Adam (16) and I'v...",Im so lostHello my name is Adam 16 and Ive bee...,in so costello my name is adam of and live bee...,im so losthello my name is adam 16 and ive bee...
...,...,...,...,...,...,...,...
232069,If you don't like rock then your not going to ...,non-suicide,0,If you don't like rock then your not going to ...,If you dont like rock then your not going to g...,if you done like rock then your not going to g...,if you dont like rock then your not going to g...
232070,You how you can tell i have so many friends an...,non-suicide,0,You how you can tell i have so many friends an...,You how you can tell i have so many friends an...,you how you can tell i have so many friends an...,you how you can tell i have so many friends an...
232071,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide,0,pee probably tastes like salty tea!! can someo...,pee probably tastes like salty tea can someone...,pee probably tastes like salty tea can someone...,pee probably tastes like salty tea can someone...
232072,The usual stuff you find hereI'm not posting t...,suicide,1,The usual stuff you find hereI'm not posting t...,The usual stuff you find hereIm not posting th...,the usual stuff you find herein not posting th...,the usual stuff you find hereim not posting th...


### Tokenization

#### Using CountVectorizer()

In [ ]:
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

In [ ]:
predictors = 'conv_emojis'
outcome = 'encoded_class'

X_t = data_df[predictors]
y = data_df[outcome]

X_train, X_test, y_train, y_test = train_test_split(X_t, y, test_size=0.3, random_state=1)

vectorizer = TfidfVectorizer(encoding='utf-8', lowercase=True, 
                             stop_words={'english'}, strip_accents='unicode',
                             analyzer='word', tokenizer=LemmaTokenizer())

# took 3 minutes
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

#mnb = make_pipeline(CountVectorizer(), TfidfTransformer(), MultinomialNB())

print(X_train.shape)
print(y_train.shape)

(162451, 168574)
(162451,)


#### Using NLTK modules

In [ ]:
def convert_lowercase(df, column):
    """

    :param df:
    :return:
    """
    df['lower_text'] = df[column].apply(lambda x: " ".join(x.lower() for x in x.split()))
    return df

In [ ]:
def get_all_words(cleaned_tokens_list):
    """

    :param cleaned_tokens_list:
    :return:
    """
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [ ]:
def remove_noise(text_tokens, stop_words):
    """

    :param text_tokens:
    :param stop_words:
    :return:
    """
    cleaned_tokens = []

    for token, tag in pos_tag(text_tokens):
        token = re.sub("_", " ", token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token not in stop_words:
            cleaned_tokens.append(token)

    return cleaned_tokens

In [ ]:
def get_text_for_model(cleaned_tokens_list):
    for text_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in text_tokens)

In [ ]:
%%script false
data_df = convert_lowercase(data_df, 'conv_emojis')

In [ ]:
%%script false
s_text_tokens = data_df.query('encoded_class == 1')['lower_text'].apply(word_tokenize)
ns_text_tokens = data_df.query('encoded_class == 0')['lower_text'].apply(word_tokenize)

In [ ]:

stop_words = stopwords.words('english')

s_cleaned_tokens = []
ns_cleaned_tokens = []

for s_row_tokens in s_text_tokens:
  s_cleaned_tokens.append(remove_noise(s_row_tokens, stop_words))

for ns_row_tokens in ns_text_tokens:
  ns_cleaned_tokens.append(remove_noise(ns_row_tokens, stop_words))

In [ ]:
def save_tokens_to_file(filename, list_tokens):
  with open(data_path + filename, "w") as txt_file:
    for line in list_tokens:
        txt_file.write(" ".join(line) + "\n") # works with any number of elements in a line

In [ ]:
def convert_file_to_tokens(filename):
  with open(data_path + filename) as f:
    contents = f.readlines()

  cleaned_tokens = []

  for each_line in contents:
    each_line = each_line.replace('\n', '')
    cur_line_list = each_line.split(' ')
    cleaned_tokens.append(cur_line_list)

  return cleaned_tokens

In [ ]:
save_tokens_to_file("suicide_intent_tokens.txt", s_cleaned_tokens)
save_tokens_to_file("non_suicide_intent_tokens.txt", ns_cleaned_tokens)

In [ ]:
f_s_cleaned_tokens = convert_file_to_tokens("suicide_intent_tokens.txt")
f_ns_cleaned_tokens = convert_file_to_tokens("non_suicide_intent_tokens.txt")

In [ ]:
f_s_cleaned_tokens

In [ ]:
s_all_words = get_all_words(f_s_cleaned_tokens)
ns_all_words = get_all_words(f_ns_cleaned_tokens)

In [ ]:
freq_dist_pos = FreqDist(s_all_words)
print(freq_dist_pos.most_common(10))

[('im', 278502), ('dont', 179556), ('want', 160355), ('get', 152756), ('go', 141402), ('like', 134654), ('feel', 133365), ('know', 124837), ('life', 114743), ('ive', 97417)]


In [ ]:
freq_dist_neg = FreqDist(ns_all_words)
print(freq_dist_neg.most_common(10))

[('im', 62808), ('filler', 54239), ('like', 53648), ('get', 48500), ('dont', 34612), ('go', 31605), ('know', 28899), ('want', 27786), ('say', 27368), ('fuck', 26701)]


In [ ]:
s_tokens_for_model = get_text_for_model(f_s_cleaned_tokens)
ns_tokens_for_model = get_text_for_model(f_ns_cleaned_tokens)

positive_dataset = [(tweet_dict, "Suicide")
                    for tweet_dict in s_tokens_for_model]

negative_dataset = [(tweet_dict, "Non-Suicide")
                    for tweet_dict in ns_tokens_for_model]

In [ ]:
dataset = positive_dataset + negative_dataset

In [ ]:
len(dataset)

232074

#### Using Spacy

In [ ]:
def spacy_preprocessing(text, nlp):
    """
    Method to preprocess data - removes stop words, lemmatizes, removes punctuation, removes digits
    and only allowed english characters
    :param text: (String) a string that needs to be preocessed
    :param nlp: (Spacy) a spacy object loaded with only english dictionary
    :return: (String) a processed string
    """
    text = unidecode(text)
    text = text.lower()
    text = re.sub("\n", " ", text)

    doc = nlp(text)
    lemmatized_stopped_list = []

    for token in doc:
        if token.is_stop is False and len(token.lemma_) > 1:
            lemmatized_stopped_list.append(token.lemma_)

    tokenized_text = " ".join(lemmatized_stopped_list).strip()
    tokenized_text = tokenized_text.translate(str.maketrans('', '', string.punctuation))
    tokenized_text = tokenized_text.translate(str.maketrans('', '', string.digits))
    tokenized_text = ' '.join(tokenized_text.split())

    return tokenized_text

## Benchmark - Bag of Words

### sklearn Naive Bayes Classifier

In [ ]:
clf = MultinomialNB().fit(X_train, y_train)

In [ ]:
predicted = clf.predict(X_test)
print(f"Accuracy is: {np.mean(predicted == y_test)}")

# Accuracy is: 0.8782155322235469

Accuracy is: 0.8782155322235469


In [ ]:
X_train

<162451x168574 sparse matrix of type '<class 'numpy.float64'>'
	with 11704522 stored elements in Compressed Sparse Row format>

### NLTK Naive Bayes Classifier

In [ ]:
random.shuffle(dataset)

In [ ]:
n_train = int(0.7 * len(dataset))

train_data = dataset[0:n_train]
test_data = dataset[n_train:]

In [ ]:
classifier = NaiveBayesClassifier.train(train_data)
print(f"Accuracy is: {classify.accuracy(classifier, test_data)}")

# Accuracy is: 0.5596713729658302

Accuracy is: 0.5608204185398503


In [ ]:
train_data

In [ ]:
print(classifier.show_most_informative_features(30))

Most Informative Features
                anymorei = True           Suicid : Non-Su =    660.9 : 1.0
                 myselfi = True           Suicid : Non-Su =    253.5 : 1.0
               anymoreim = True           Suicid : Non-Su =    250.7 : 1.0
                   helpi = True           Suicid : Non-Su =    221.1 : 1.0
         hospitalization = True           Suicid : Non-Su =    150.8 : 1.0
                      sw = True           Suicid : Non-Su =    150.8 : 1.0
                  helpim = True           Suicid : Non-Su =    137.5 : 1.0
                    itim = True           Suicid : Non-Su =    136.4 : 1.0
               suicidali = True           Suicid : Non-Su =    132.8 : 1.0
                   itive = True           Suicid : Non-Su =    124.2 : 1.0
                  lifeim = True           Suicid : Non-Su =    124.2 : 1.0
                ideation = True           Suicid : Non-Su =    105.7 : 1.0
                  alonei = True           Suicid : Non-Su =    102.9 : 1.0

# Using a pretrained BERT model

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 16.1 MB/s 
     |████████████████████████████████| 6.6 MB 40.7 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Preprocessing model